# Sentinel-2 imagery access and analysis

Since we are using Jupyter notebook, we will access the data right here using sentinelsat a python library which makes searching, retrieving and downloading Sentinel satellite images easy.

##### Dependencies:
conda install -c conda-forge jupyterlab
<br>conda install -c conda-forge sentinelsat
<br>conda install geopandas
<br>conda install rasterio
<br>conda install -c conda-forge folium
<br>conda install -c conda-forge ipywidgets

Before we are able to use sentinelsat, we need to register a username in Copernicus Open Access Hub and note down your username and password and paste them here inside the code

In [ ]:
import rasterio as rio

In [ ]:
from sentinelsat import SentinelAPI

user = 'username' 
password = 'password' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

With the code below, we read the area of interest (AOI) shapefile in Geopandas and call it AOI, then later create an empty base map in Folium centred around coordinates in the area, we call this m. Finally, we can add the Geopandas data to the base map we have created to visualize the AOI boundary we are interested in. Below you can see the map.

In [ ]:
import geopandas as gpd
import folium 

AOI = gpd.read_file('shapefile_name.shp')

m = folium.Map([lat, lon], zoom_start=12)
folium.GeoJson(AOI).add_to(m)
m

One last step before we can search and download sentinel 2 images is to create a footprint from the AOI geometry. Here we will use Shapely Python library since our data is in Shapefiles and have read it already as Geopandas GeodataFrame. (Note that if you have Geojson data, sentinelsat provides a handy way to convert your data into a proper format in the query).

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in AOI['geometry']:
    footprint = i

Now we can run a query on the api we have created above. There are different ways you can construct your query here depending on your use case. In this example, we will create a query for Sentinel 2 images Level 2A with cloud coverage between 0 and 10 that fall or intersect with the footprint (Area of study). For the time period, we are interested only in Sentinel Level 2A satellite images (For reference on valid search queries please refer to scihub).

In [ ]:
products = api.query(footprint,
                     date = ('20210801', '20210810'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10)
                    )

We get a dictionary of all products available in this period with the query specification. You can tweak the query for your use case example expanding the time period or increasing the cloud coverage percentage. From here we can create a GeodataFrame or Dataframe from the product dictionary and sort them according to cloud coverage percentage. GeodataFrame holds the geometry of each satellite image tile. Once we create the GeodataFrame and sort it, we call directly products_gdf_sorted table to see the attributes off all rows.

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted

The table above only shows the first columns of the products_gdf_sorted table. In the index, you have tiles which you can use to download the particular image. Additional columns also include a title which has the full name of the tile and some other useful columns like cloud coverage percentage. Let's say we are interested in the first satellite image. We can simply call download and provide the product name (Note that you can download all images at once with api.download_all() function).

In [ ]:
api.download("7dbb26e5-c5e3-484a-a5b4-c4538d7c2cf2")

This will take a while (Sentinel 2 Satellite image tiles are about 1 GB). Once the download is finished, we can simply unzip it. In the next section, we will use the downloaded satellite images to process, analyze and visualize them.

## Exploring Imagery with Rasterio

Once we unzip the downloaded folder, we get many subfolders and it is sometimes hard to navigate through these folders. Here is a tree of the folders.

![title](tree_example.png)

Sentinel-2 data is multispectral with 13 bands in the visible, near infrared and shortwave infrared spectrum. These bands come in a different spatial resolution ranging from 10 m to 60 m, thus images can be categorized as high-medium resolution. While there are other higher resolution satellites available(1m to 0.5 cm), Sentinel-2 data is free and has a high revisit time (5 days) which makes it an excellent option to study environmental challenges.

### Create RGB Image

The true colour of satellite images is often displayed in a combination of the red, green and blue bands. Let us first read the data with Rasterio and create an RGB image from Bands 4, 3, and 2.
First, we open an empty RGB.tiff in Rasterio with the same parameters i.e. width, height, CRS, etc. Then we need to write those bands to the empty RGB image.

In [ ]:
# Open Bands 4, 3 and 2 with Rasterio
R10 = r'C:\Users\....\S2A_MSIL2A_20210802T075611_N0301_R035_T37SEV_20210802T110235.SAFE\GRANULE\L2A_T37SEV_A031921_20210802T080227\IMG_DATA/R10m'
b4 = rio.open(R10+'/T37SEV_20210802T075611_B04_10m.jp2')
b3 = rio.open(R10+'/T37SEV_20210802T075611_B03_10m.jp2')
b2 = rio.open(R10+'/T37SEV_20210802T075611_B02_10m.jp2')


# Create an RGB image 
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

### Create NDVI

Calculating Normalized Difference Vegetation Index (NDVI) is an important indicator to assess the presence/absence of green vegetation from the satellite images. To calculate the NDVI, we need Red band and Near-Infrared Band (NIR). Different satellite images assign different numbers for these bands. Sentinel Images have red in 4th band and NIR in the 8th band. The formula for NDVI calculation is:

#### NIR - RED /(NIR + RED)

To carry out this in Rasterio we need first to read the 4th and 8th bands as arrays. We also need to make sure that the arrays are floats.

In [ ]:
# Open b4 and b8
b4 = rio.open(R10+'/T37SEV_20210802T075611_B04_10m.jp2')
b8 = rio.open(R10+'/T37SEV_20210802T075611_B08_10m.jp2')

# read Red(b4) and NIR(b8) as arrays
red = b4.read()
nir = b8.read()

# Calculate ndvi
ndvi = (nir.astype(float)-red.astype(float))/(nir+red)

# Write the NDVI image
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('NDVI.tif', 'w', **meta) as dst:
    dst.write(ndvi.astype(rio.float32))

The output is an NDVI image which shows vegetation level of areas in the satellite image.

Accessing Sentinel 2 images with Python is made easy with sentinelsat. In this tutorial, we have covered how to construct a query and retrieve on information from available images as well as how to download Sentinel 2 images within Jupyter notebooks. We have also seen how to preprocess, create RGB and NDVI images and visualize raster images with Rasterio. 